In [2]:
import os
import json
import time
import random
import lxml
import requests
import json
import time
import sys
import re
import os
from requests.exceptions import RequestException
from bs4 import BeautifulSoup as bsoup
import random
import pandas as pd

In [3]:
from contextlib import closing
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException

In [19]:

delhi_phd = pd.read_html("http://www.cse.iitd.ac.in/index.php/2011-12-29-23-14-30/alumni/phd-alumni", header=0, parse_dates=['Year'])

In [ ]:
delhi_faculty=pd.read_html("http://www.cse.iitd.ac.in/index.php/2011-12-29-23-14-30/faculty",skiprows=[0,2])

In [6]:
def simple_get(url):
    try:
        with closing(requests.Session().get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    print(e)

In [12]:
delhi_phd[0].values[1:5]

array([['Neha Sengupta',
        'Algorithms For Massive And Dynamic Graph Data Management',
        Timestamp('2019-01-01 00:00:00'),
        'Research Associate at Inception Institute of Artificial Intelligence, Abu Dhabi UAE'],
       ['Suvam Patra',
        'Robust Scene Geometry Recovery From Ego-Cameras For Visual Navigation',
        Timestamp('2019-01-01 00:00:00'),
        'Research Engineer at Scape Technologies'],
       ['Ankit Anand',
        'Symmetry Aware Inference And Decision Making In Probabilistic Models',
        Timestamp('2019-01-01 00:00:00'),
        'Research Scientist at Google DeepMind Montreal'],
       ['Jatin Batra', 'Dynamic Programming For Scheduling Problems',
        Timestamp('2019-01-01 00:00:00'), 'Post Doc at CWI, Amsterdam']],
      dtype=object)

In [7]:
#url='http://eprint.iitd.ac.in/browse?type=title'
supervisor_all=[]
subject_all=[]
inst_all=[]
key_all=[]
name_all=[]
base_url='http://eprint.iitd.ac.in'
for t in delhi_phd[0]['Thesis Title'].values:
    thesis=t
    search_url="http://eprint.iitd.ac.in/discover?scope=2074%2F7856&&query={0}&submit=".format(thesis)
    search_item=simple_get(search_url)
    if search_item is not None:
        supervisor=[]
        subject=[]
        inst=[]
        key=[]
        name=[]
        search_item_soup=bsoup(search_item,'html.parser')
        handle=search_item_soup.find('div', attrs={'class':"col-sm-9 artifact-description"})
        if handle is not None:
            handle=handle.find('a',href=True)['href']
            url=base_url+handle
            time.sleep(random.randint(20,25))
            search_item2=simple_get(url)
            if search_item2 is not None:
                search_item2_soup=bsoup(search_item2,'html.parser')
                content=search_item2_soup.find_all("meta")
                for a in content:
                    if a.attrs.get('name')=="DC.contributor":
                        supervisor.append(a['content'])
                    elif a.attrs.get('name')=="DC.subject":
                        subject.append(a['content'])
                    elif a.attrs.get('name')=="citation_dissertation_institution":
                        inst.append(a['content'])
                    elif a.attrs.get('name')=="citation_keywords":
                        key.append(a['content'])
                    elif a.attrs.get('name')=="DC.creator":
                        name.append(a['content'])
                    else:
                        continue
    supervisor_all.append(supervisor)
    subject_all.append(subject)
    inst_all.append(inst)
    key_all.append(key)
    name_all.append(name)

In [20]:
delhi_phd=delhi_phd[0]

In [21]:
len(delhi_phd)

104

In [22]:
delhi_phd['guide']=["|".join(a).replace('\t',' ') for a in supervisor_all]
delhi_phd['subject']=["|".join(a).replace('\t',' ') for a in subject_all]
delhi_phd['inst']=["|".join(a).replace('\t',' ') for a in inst_all]
delhi_phd['key']=["|".join(a).replace('\t',' ') for a in key_all]
delhi_phd['student']=["|".join(a).replace('\t',' ') for a in name_all]
delhi_phd.to_csv('college/iit_delhi.csv', sep='\t', index=False, escapechar='/')

In [37]:
res=0
for a, b in zip(name_all,supervisor_all):
    if len(a) != 0 and len(b) != 0:
        res+=1
    else:
        continue

In [40]:
flat_list = [item for sublist in supervisor_all for item in sublist]